## Rerun imports and initialization

In [1]:
from collections import namedtuple
from math import cos, sin, tau
import math

import numpy as np
import rerun as rr  # pip install rerun-sdk
import rerun.blueprint as rrb

rr.init("rerun_example_cube")

## Optional: start a local web-viewer server

By default, Rerun will use a copy of the viewer hosted at [https://app.rerun.io](https://app.rerun.io).
This is generally preferable as it will work more seamlessly even if you
are connected to a notebook instance on a remote machine. However there
are some cases where this won't work such as running from source, or
using your notebook in an offline environment.

In these cases you can start a local viewer server by uncommenting the following
line:

In [10]:
# rr.start_web_viewer_server()

## Helper to create the colored cube

This code exists in the `rerun.utilities` package, but is repeated here for context.

In [3]:
ColorGrid = namedtuple("ColorGrid", ["positions", "colors"])


def build_color_grid(x_count=10, y_count=10, z_count=10, x_size=10, twist=0):
    """
    Create a cube of points with colors.

    The total point cloud will have x_count * y_count * z_count points.

    Parameters
    ----------
    x_count, y_count, z_count:
        Number of points in each dimension.
    twist:
        Angle to twist from bottom to top of the cube

    """

    grid = np.mgrid[
        slice(-x_count, x_count, x_count * 1j),
        slice(-y_count, y_count, y_count * 1j),
        slice(-z_count, z_count, z_count * 1j),
    ]

    angle = np.linspace(-float(twist) / 2, float(twist) / 2, z_count)
    for z in range(z_count):
        xv, yv, zv = grid[:, :, :, z]
        rot_xv = xv * cos(angle[z]) - yv * sin(angle[z])
        rot_yv = xv * sin(angle[z]) + yv * cos(angle[z])
        grid[:, :, :, z] = [rot_xv, rot_yv, zv]

    positions = np.vstack([xyz.ravel() for xyz in grid])

    colors = np.vstack(
        [
            xyz.ravel()
            for xyz in np.mgrid[
                slice(0, 255, x_count * 1j),
                slice(0, 255, y_count * 1j),
                slice(0, 255, z_count * 1j),
            ]
        ]
    )

    return ColorGrid(positions.T, colors.T.astype(np.uint8))

## Logging some data

Now we can create some data and add it to the recording.

In [4]:
rr.init("rerun_example_cube")
STEPS = 100
twists = math.pi * np.sin(np.linspace(0, math.tau, STEPS)) / 4
for t in range(STEPS):
    rr.set_time_sequence("step", t)
    cube = build_color_grid(10, 10, 10, twist=twists[t])
    rr.log("cube", rr.Points3D(cube.positions, colors=cube.colors, radii=0.5))

## Viewing the output

The current stream can be output to the cell using `rr.notebook_show()`

In [5]:
rr.notebook_show()

## Adjusting the view

The  `show` method also lets you adjust properties such as width and height.

In [6]:
rr.notebook_show(width=400, height=400)

## Stating a new recording

You can always start another recording by calling `rr.init(...)` again.

In [7]:
rr.init("rerun_example_cube")

STEPS = 100
twists = math.pi * np.sin(np.linspace(0, math.tau, STEPS)) / 4
for t in range(STEPS):
    rr.set_time_sequence("step", t)
    h_grid = build_color_grid(10, 3, 3, twist=twists[t])
    rr.log("h_grid", rr.Points3D(h_grid.positions, colors=h_grid.colors, radii=0.5))
    v_grid = build_color_grid(3, 3, 10, twist=twists[t])
    rr.log("v_grid", rr.Points3D(v_grid.positions, colors=v_grid.colors, radii=0.5))

rr.notebook_show()

## Using blueprints

Rerun blueprints can be used with `rr.show()`

For example, we can split the two grids into their own respective space-views.

In [8]:
blueprint = rrb.Blueprint(
    rrb.Horizontal(
        rrb.Spatial3DView(name="Horizontal grid", origin="h_grid"),
        rrb.Spatial3DView(name="Vertical grid", origin="v_grid"),
        column_shares=[2,1]),
    collapse_panels=True
)

rr.notebook_show(blueprint=blueprint)

## Extra convenience

Rerun blueprints types also implement `_repr_html_()` directly, so if a blueprint is the last element in your cell the right thing will happen.

In [9]:
rrb.Spatial3DView(name="Horizontal grid", origin="h_grid")